<a href="https://colab.research.google.com/github/Pyfin5/Machine_Learning/blob/main/Reddit_Poutine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Colab-Notebooks/Reddit_listening/

/content/drive/MyDrive/Colab-Notebooks/Reddit_listening


Very important to have this, otherwise pip dependencies will take forever to install due to the older dependency resolver.

In [3]:
!pip install --upgrade pip

List of libraries that you'll need.

In [4]:
!pip install \
    praw \
    pandas \
    numpy \
    python-dotenv \
    datetime \
    IPython \
    llama_index \
    langchain_google_vertexai \
    google-cloud-aiplatform \
    praw \
    textwrap3 \
    langextract \
    asyncpraw

Importing libraries

In [5]:
import praw
import pandas as pd
import os
import numpy as np
from dotenv import load_dotenv
import datetime as dt
import pandas as pd  # Data manipulation library
import sys
from IPython.display import Markdown, display
import pandas as pd
from dotenv import load_dotenv
from langchain_google_vertexai import VertexAIEmbeddings
from llama_index.core.llms import ChatMessage
from google.cloud import aiplatform
from praw.models import MoreComments
import textwrap
import langextract as lx
from langextract import data
from google.colab import userdata
import asyncpraw as asyncpraw

Use load_dotenv() is you're running on an IDE, otherwise for colab use userdata.get() and extract from secrets manager.

In [6]:
load_dotenv() # Load environment variables from .env file

# Instantiate reddit API
reddit = praw.Reddit(client_id=userdata.get("client_id"),
                     client_secret=userdata.get("client_secret"),
                     user_agent=userdata.get("user_agent"),
                     username=userdata.get("username"),
                     password=userdata.get("password"))


# Subreddit to scrape
subreddit = reddit.subreddit('askTO')

# Display the name of the Subreddit
print("Display Name:", subreddit.display_name)




Display Name: askTO


Might get some warnings on using asyncpraw instead of praw library. I say we ignore this warning and march on! Or just modify the code for asyncpraw

In [8]:


submission = reddit.submission(url = "https://www.reddit.com/r/askTO/comments/1llx9cx/partner_is_upset_that_i_visit_quebec_every_2/")


# for comment in submission.comments.list():
#         comments.append({
#         'Type': 'Comment',
#         'Post_id': submission.id,
#         'Title': submission.title,
#         'Author': comment.author.name if comment.author else 'Unknown',
#         'Timestamp': comment.created_utc,
#         'Text': comment.body,
#         'Score': submission.score,
#         'Total_comments': submission.num_comments,
#         'Post_URL': submission.url
#     })
#         i += 1
#         print(i)
#         print(comments)

In [12]:
comments_df = []

submission.comments.replace_more(limit=None)

comment_queue = submission.comments[:]
while comment_queue:
    comment = comment_queue.pop(0)
    comments_df.append([comment.body, comment.score])
    comment_queue.extend(comment.replies)

In [18]:
print(len(comments_df))
print(comments_df[0])
comments_df = [comment[0].rstrip() for comment in comments_df]
print(comments_df)

369
['Don’t you have enough at home?', 400]
['Don’t you have enough at home?', 'I’m actually curious to know what kind of poutine is making you drive all the way to Quebec…', 'If I had the funds, I would go to Quebec every two months for routine and bagels.', 'Farm boy sells st.alberts curds cheese if you want to try and make it at home!', 'This post is so funny. Good for you girl - get that poutine.', 'You can try finding other options on dating apps, joining a club in your area is also a good way to meet people /s', 'I would go on a regular basis from Barrie to Montreal to eat at St Hubert, so it’s not that unusual.', 'The pantry fine cheese shipps in fresh cheese curds every friday. They are stored @ room temperature throughout the weekend until they sell out. Thats how you know its fresh. They can be stored @ room temperature for quite a few days when fresh.\nThey squeek like Quebec curds. You can make your own authentic poutine this way. Be there on a fri or sat. On sunday they se

In [19]:
system_prompt = (
    "Extract restaurant mentioned in this string. Make sure to provide all instances of restaurants mentioned in the string."
    "Each string might contain mention of multiple restaurants, extract them all including multple instances of the same restuarant."
    "Each restaurant is based in the city of Toronto so extract the area as part of attributes"
    )


examples = [
            lx.data.ExampleData(
                text= "… you’ll laugh, but a killer price/quality is Costco.I think it’s $5.99 and you get a solid 2lbs. (And yeah, it used to be poutini’s)",
                extractions=[
                    lx.data.Extraction(extraction_class="Restaurant", extraction_text="Costco", attributes = {'Top':'Yes',"location":None}),
                    lx.data.Extraction(extraction_class="Restaurant", extraction_text="poutini", attributes = {'Top':'No',"location":None})
                ]
            ),
            lx.data.ExampleData(
                text=   "It used to be Poutini’s. Oh man that was the ONE. but now I dunno… there’s a good one at The Pumps in Leslieville near where I work. So I’ll go there OR… my other choice is in that Gerrard square mall. Tropical Joe’s has a jerk chicken poutine that’s absolutely amazing.",
                extractions=[
                    lx.data.Extraction(extraction_class="Restaurant", extraction_text="The Pumps", attributes = {'Top':'Yes',"location":"Leslieville"}),
                    lx.data.Extraction(extraction_class="Restaurant", extraction_text="Tropical Joe’s", attributes = {'Top':'No',"location":None}),
                    lx.data.Extraction(extraction_class="Restaurant", extraction_text="Gerrard square mall", attributes = {'Top':'No',"location":None}),
                    lx.data.Extraction(extraction_class="Restaurant", extraction_text="Poutini’s", attributes = {'Top':'Yes',"location":None})
                ]
            ),
            lx.data.ExampleData(
                text=   "NomNomNom poutine on Dundas is delish.",
                extractions=[
                    lx.data.Extraction(extraction_class="Restaurant", extraction_text='NomNomNom' , attributes = {'Top':'Yes',"location":"Dundas"})
                ]
            ),
            lx.data.ExampleData(
                text=   "The chip truck at the Leslie and lakeshore CT",
                extractions=[
                    lx.data.Extraction(extraction_class="Restaurant", extraction_text='the chip truck' , attributes = {'Top':'Yes',"location":"Leslie and Lakeshore CT"})
                ]
            ),
            lx.data.ExampleData(
                text=   "The best poutine in the city is from the Ideal Catering truck (aka brown and cream truck) on St. George, just south of Harbord. They have the best fries and the best gravy.",
                extractions=[
                    lx.data.Extraction(extraction_class="Restaurant", extraction_text='Ideal Catering Truck' , attributes = {'Top':'Yes',"location":"St. George"})
                ]
            ),
            lx.data.ExampleData(
                text=   "Rudy’s. I never tried NomNomNom Poutine but that looks legit also.",
                extractions=[
                    lx.data.Extraction(extraction_class="Restaurant", extraction_text="Rudy" , attributes = {'Top':'Yes',"location":None})
                ]
            ),
            lx.data.ExampleData(
                text=   "Shawarma Istanbul on College at Bathurst. Perfect fries, gravy is on point, they don’t skimp on the cheese, and the box is so full that they have to smoosh it to get it closed",
                extractions=[
                    lx.data.Extraction(extraction_class="Restaurant", extraction_text="Shawarma Istanbul" , attributes = {'Top':'Yes',"location":"College at Bathurst"})
                ]
            )
        ]

In [20]:
api_key = userdata.get('GOOGLE_AI_STUDIO_API_KEY')

In [46]:
def extract_entity(input_text, NER_examples, system_prompt, api_key):

        # Define example data with entities in order of appearance
        examples = NER_examples

        result = lx.extract(
            text_or_documents=input_text,
            prompt_description=system_prompt,
            examples=NER_examples,
            model_id="gemini-2.5-pro",
            api_key=api_key
        )

        # Display entities with positions
        print(f"Input: {input_text}\n")
        print("Extracted entities:")

        result_append =  []

        for entity in result.extractions:
            if entity.attributes != None:

              position_info = ""
              if entity.char_interval:
                  start, end = entity.char_interval.start_pos, entity.char_interval.end_pos
                  position_info = f" (pos: {start}-{end})"
              print(f"• {entity.extraction_class.capitalize()}: {entity.extraction_text}{position_info}")
              print(f"  Attributes: {entity.attributes}")
              result_append.append([entity.extraction_text, entity.attributes, input_text])

            else:
              result_append.append([None, None, input_text ])
        return result_append




Option 1 - Combine all comments in 1 list

Use the EOS token to demarcate end of each comment

In [47]:
comments_list = '<EOS>'.join(comments_df)
comments_list = comments_list.replace('\n','').replace("'","")
comments_list = comments_list.lower()


print(comments_list)

don’t you have enough at home?<eos>i’m actually curious to know what kind of poutine is making you drive all the way to quebec…<eos>if i had the funds, i would go to quebec every two months for routine and bagels.<eos>farm boy sells st.alberts curds cheese if you want to try and make it at home!<eos>this post is so funny. good for you girl - get that poutine.<eos>you can try finding other options on dating apps, joining a club in your area is also a good way to meet people /s<eos>i would go on a regular basis from barrie to montreal to eat at st hubert, so it’s not that unusual.<eos>the pantry fine cheese shipps in fresh cheese curds every friday. they are stored @ room temperature throughout the weekend until they sell out. thats how you know its fresh. they can be stored @ room temperature for quite a few days when fresh.they squeek like quebec curds. you can make your own authentic poutine this way. be there on a fri or sat. on sunday they sell out. there might be a few early sunday

In [57]:
result_generator = lx.extract(
    text_or_documents = comments_list,
    prompt_description = system_prompt,
    examples = examples,
    model_id = "gemini-2.5-flash",
    extraction_passes = 1,
    api_key = api_key
    )


We create 2 lists, appending results and a restaurant only list which we will use to create a cluster.

In [58]:
result_append =  []
restaurant_only = []

for entity in result_generator.extractions:

              position_info = ""
              if entity.char_interval:
                  start, end = entity.char_interval.start_pos, entity.char_interval.end_pos
                  position_info = f" (pos: {start}-{end})"
              print(f"• {entity.extraction_class.capitalize()}: {entity.extraction_text}{position_info}")
              print(f"  Attributes: {entity.attributes}")
              if entity.attributes != None:
                result_append.append([entity.extraction_text.strip('\n'), entity.attributes['location'],position_info])
                restaurant_only.append(entity.extraction_text.strip('\n'))
              else:
                result_append.append([entity.extraction_text.strip('\n'), None, None])
                restaurant_only.append(entity.extraction_text.strip('\n'))


• Restaurant: st hubert (pos: 549-558)
  Attributes: {'Top': 'Yes', 'location': 'montreal'}
• Restaurant: nomnomnom (pos: 1535-1544)
  Attributes: {'Top': 'Yes', 'location': 'Dundas'}
• Restaurant: leslieville pumps (pos: 1583-1600)
  Attributes: {'Top': 'Yes', 'location': 'Leslieville'}
• Restaurant: new york fries (pos: 1652-1666)
  Attributes: {'Top': 'Yes', 'location': 'Toronto'}
• Restaurant: obq burger (pos: 1847-1857)
  Attributes: {'Top': 'Yes', 'location': 'browns line off the 427 exit'}
• Restaurant: rudy (pos: 4174-4178)
  Attributes: {'Top': 'Yes', 'location': 'toronto'}
• Restaurant: Harvey’s (pos: 5003-5011)
  Attributes: {'Top': 'Yes', 'location': 'null'}
• Restaurant: Costco (pos: 5015-5021)
  Attributes: {'Top': 'Yes', 'location': 'null'}
• Restaurant: Nom Nom (pos: 5081-5088)
  Attributes: {'Top': 'Yes', 'location': 'null'}
• Restaurant: food truck (pos: 5312-5322)
  Attributes: {'Top': 'Yes', 'location': 'null'}
• Restaurant: nomnom nom (pos: 5415-5425)
  Attributes:

Create embeddings for each location, since we're unable to parse each location and add separately

In [59]:
from google import genai
from google.genai import types


client = genai.Client(api_key = api_key)



def embed_func(text):
  result = client.models.embed_content(
          model="gemini-embedding-001",
          contents = text,
          config=types.EmbedContentConfig(task_type="clustering",
                                          output_dimensionality= 768))
  return result

In [72]:
print(result_append[0])
print(len(result_append))

['st hubert', 'montreal', ' (pos: 549-558)']
144


In [63]:
restaurant_embed = list(map(embed_func, restaurant_only))
print("Using the map() function:", restaurant_embed)

Using the map() function: [EmbedContentResponse(
  embeddings=[
    ContentEmbedding(
      values=[
        -0.010977463,
        -0.0073278123,
        0.00091233966,
        -0.05951305,
        -0.0045933104,
        <... 763 more items ...>,
      ]
    ),
  ],
  sdk_http_response=HttpResponse(
    headers=<dict len=10>
  )
), EmbedContentResponse(
  embeddings=[
    ContentEmbedding(
      values=[
        -0.022119652,
        0.027757557,
        -0.0012565439,
        -0.06891831,
        0.0013441693,
        <... 763 more items ...>,
      ]
    ),
  ],
  sdk_http_response=HttpResponse(
    headers=<dict len=10>
  )
), EmbedContentResponse(
  embeddings=[
    ContentEmbedding(
      values=[
        -0.005683687,
        0.006683339,
        0.0030247872,
        -0.05746787,
        -0.0016343471,
        <... 763 more items ...>,
      ]
    ),
  ],
  sdk_http_response=HttpResponse(
    headers=<dict len=10>
  )
), EmbedContentResponse(
  embeddings=[
    ContentEmbedding(

In [65]:
len(restaurant_embed)
#print(restaurant_embed[0])

144

In [73]:
combined_list = [(restaurant_only[i], restaurant_embed[i].embeddings[0].values, result_append[i][2]) for i in range(len(restaurant_only))]

print(len(combined_list))
print(combined_list[0:10])

144
[('st hubert', [-0.010977463, -0.0073278123, 0.00091233966, -0.05951305, -0.0045933104, 0.0037570957, 0.01054967, 0.009326226, -0.0027473902, 0.010303406, -0.0016060665, -0.038068518, -0.0099413805, 0.005259274, 0.08753557, -0.018641785, -0.005911679, 0.0076792454, 0.0076305484, -0.016873766, 0.0056405715, 0.0008839351, -0.0008474599, -0.011227529, -0.014575472, 0.013323022, 0.01233536, 0.03315994, 0.049360536, 0.040148776, -0.00045367144, 0.014449646, -0.0019936408, 0.0059625222, 0.0011821098, 0.013541547, 0.013640884, 0.0010782626, 0.013231937, 7.9116806e-05, -0.0023579597, 0.015215703, 0.028111521, 0.026215255, -0.0072676996, -0.01355896, 0.0149574, -0.030804668, -0.0023355098, 0.015445746, 0.0018789608, -0.026608435, -0.0075844657, -0.16949055, 0.018583285, 0.0033255664, -0.0054531796, -0.0044566947, 0.011879905, -0.011791381, -0.018739475, 0.0189624, 0.012474429, -0.022715462, 0.0017896192, 0.021745035, 0.0231787, 0.0217733, -0.0069812885, 0.0136792455, 0.011836333, -0.0021359

In [81]:
embed_df = pd.DataFrame(combined_list, columns=['restaurant', 'embedding','POS'])
print(embed_df)

             restaurant                                          embedding  \
0             st hubert  [-0.010977463, -0.0073278123, 0.00091233966, -...   
1             nomnomnom  [-0.022119652, 0.027757557, -0.0012565439, -0....   
2     leslieville pumps  [-0.005683687, 0.006683339, 0.0030247872, -0.0...   
3        new york fries  [-0.010235135, -0.011222426, 0.0058330977, -0....   
4            obq burger  [-0.0041924594, -0.01929776, 0.0048127985, -0....   
..                  ...                                                ...   
139  poutinerie trucker  [-0.009902984, -0.0017131383, 0.018598748, -0....   
140        casse-croute  [-0.0057440395, 0.018317731, -0.0006934585, -0...   
141             lafleur  [-0.011979939, -0.005779488, -0.017923072, -0....   
142      belle province  [-0.0036311126, 5.171525e-06, 0.0019443249, -0...   
143        high poutine  [0.0031732447, 0.0041970047, -0.005655673, -0....   

              POS start_POS end_POS  
0         549-558       5

After creating our embdedings, we get to 3,072 dimensions. In order to visualize this, we need to bring it down to 2 or 3. We use TSNE for this.

In [82]:
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import numpy as np

tsne = TSNE(random_state=0, n_iter=1000)
tsne_results = tsne.fit_transform(np.array(embed_df['embedding'].tolist()))
df_tsne = pd.DataFrame(tsne_results, columns=['TSNE1', 'TSNE2'])
df_tsne['reddit_restaurant_name'] = embed_df['restaurant']

/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


We now use KMeans algorithm to create clusters which we will use for plotting

In [83]:
kmeans_model = KMeans(n_clusters=20, random_state=1, n_init='auto').fit(np.array(embed_df['embedding'].tolist()))
labels = kmeans_model.fit_predict(np.array(embed_df['embedding'].tolist()))

In [84]:
df_tsne['cluster'] = labels

In [87]:
import plotly.express as px
fig = px.scatter(df_tsne, x='TSNE1', y='TSNE2', color = 'cluster', hover_data=['reddit_restaurant_name'])
fig.show()

In [86]:
!pip install chart-studio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [chart-studio]


In [ ]:
df_tsne.to_csv('tsne)_results.csv', index=False)